In [1]:
from src.dysregnet import dysregnet
import pandas as pd
import pickle
import numpy as np
import os

Get data

In [2]:
current_dir = os.getcwd()
grn_breast = pd.read_csv(current_dir + "/GENIE3_output/linkedList_output_slurm_gene_tpm_v10_breast_mammary_tissu_filtered.csv")
grn_lung = pd.read_csv(current_dir + "/GENIE3_output/linkedList_output_slurm_gene_tpm_v10_lung_filtered.csv")

exp_lung = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_lung_filtered.csv")
exp_breast = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_breast_mammary_tissu_filtered.csv")

In [3]:
grn_exercise = pd.read_csv("/exercise_data/regulatory_network.csv")
exp_exerise = pd.read_csv("/exercise_data/LUAD_expression.csv")
meta_exercise = pd.read_csv("/exercise_data/LUAD_meta.csv")

In [4]:
"""meta_lung = pd.DataFrame({
    'sample': exp_lung.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_lung.columns[1:])),
    'gender' : 'FEMALE'
})
meta_lung"""

"meta_lung = pd.DataFrame({\n    'sample': exp_lung.columns[1:],\n    'condition': np.random.choice([0, 1], size=len(exp_lung.columns[1:])),\n    'gender' : 'FEMALE'\n})\nmeta_lung"

In [5]:
grn_breast

,regulatoryGene,targetGene,weight
0,PAX8,PAX8-AS1,0.149235
1,ZFY,EIF1AY,0.101827
2,ZFY,TXLNGY,0.101697
3,ZFY,XIST,0.098289
4,ZFY,KDM5D,0.098019
...,...,...,...
99995,MEIS3,SNAI2,0.014984
99996,ZNF629,SLC25A44,0.014984
99997,SPDEF,ENSG00000264067,0.014984
99998,ZNF33A,NUP107,0.014984


In [6]:
grn_breast
meta_breast = pd.DataFrame({
    'sample': exp_breast.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_breast.columns[1:])),
    'gender' : 'FEMALE'
})
meta_breast

,sample,condition,gender
0,GTEX-1117F-2826-SM-5GZXL,0,FEMALE
1,GTEX-111YS-1926-SM-5GICC,0,FEMALE
2,GTEX-1122O-1226-SM-5H113,1,FEMALE
3,GTEX-117XS-1926-SM-5GICO,1,FEMALE
4,GTEX-117YX-1426-SM-5H12H,1,FEMALE
...,...,...,...
509,GTEX-ZYT6-0126-SM-5E45J,1,FEMALE
510,GTEX-ZYW4-0826-SM-5GIDG,1,FEMALE
511,GTEX-ZYY3-2526-SM-GMXAZ,0,FEMALE
512,GTEX-ZZ64-1226-SM-5E43R,1,FEMALE


In [7]:
meta_lung= pd.DataFrame({
    'sample': exp_lung.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_lung.columns[1:])),
    'gender' : 'FEMALE'
})
meta_lung

,sample,condition,gender
0,GTEX-111CU-0326-SM-5GZXO,1,FEMALE
1,GTEX-111FC-1126-SM-5GZWU,0,FEMALE
2,GTEX-111VG-0726-SM-5GIDC,1,FEMALE
3,GTEX-111YS-0626-SM-5GZXV,0,FEMALE
4,GTEX-1122O-0126-SM-5GICA,0,FEMALE
...,...,...,...
599,GTEX-ZYVF-1726-SM-5E443,1,FEMALE
600,GTEX-ZYW4-1526-SM-5MR8B,1,FEMALE
601,GTEX-ZYY3-0926-SM-5E454,1,FEMALE
602,GTEX-ZZPT-1326-SM-5E43H,1,FEMALE


In [8]:
# Step 1: Filter rows where condition is 0 (controls)
controls = meta_breast[meta_breast['condition'] == 0]

# Step 2: Randomly select two control samples
selected_controls = controls.sample(n=2, random_state=42)  # random_state for reproducibility

# Step 3: Update the meta_breast DataFrame to only include the selected controls and all other rows where condition != 0
meta_breast_filtered = pd.concat([
    selected_controls,
    meta_breast[meta_breast['condition'] != 0]
], ignore_index=True)

# Display the updated meta_breast DataFrame
print(meta_breast_filtered)


                       sample  condition  gender
0     GTEX-XUZC-1626-SM-4BRVP          0  FEMALE
1    GTEX-11P82-1326-SM-5HL62          0  FEMALE
2    GTEX-1122O-1226-SM-5H113          1  FEMALE
3    GTEX-117XS-1926-SM-5GICO          1  FEMALE
4    GTEX-117YX-1426-SM-5H12H          1  FEMALE
..                        ...        ...     ...
255   GTEX-ZY6K-1626-SM-5GZWV          1  FEMALE
256   GTEX-ZYFC-0826-SM-5E44K          1  FEMALE
257   GTEX-ZYT6-0126-SM-5E45J          1  FEMALE
258   GTEX-ZYW4-0826-SM-5GIDG          1  FEMALE
259   GTEX-ZZ64-1226-SM-5E43R          1  FEMALE

[260 rows x 3 columns]


In [9]:
grn_lung

,regulatoryGene,targetGene,weight
0,ZFY,TTTY14,0.269712
1,ZFY,EIF1AY,0.248673
2,ZFY,UTY,0.244710
3,ZFY,TXLNGY,0.240938
4,ZFY,USP9Y,0.239011
...,...,...,...
99995,ZNF571,PRPSAP1,0.014408
99996,L3MBTL1,CERS3-AS1,0.014408
99997,TWIST1,DIO2,0.014408
99998,DMRTA1,MAGI3,0.014408


Define the confounding variables or the design matrix

In [10]:
# define condition column (0 indicated control, 1 indicates case), Column name for the condition in the meta DataFrame.
conCol='condition'

# define categorical confounder columns in meta dataframe, List of categorical variable names. They should match the name of their columns in the meta Dataframe.
CatCov=['gender']  

# define continuous confounder columns in meta dataframe, List of continuous covariates. They should match the name of their columns in the meta Dataframe.
# ConCov=['gender']

In [11]:
exp_lung = exp_lung.set_index(exp_lung.columns[0])
exp_lung = exp_lung.T 
exp_lung.insert(0, "sample", exp_lung. index)
#assert all(exp_lung.iloc[:, 0].values == meta_lung.iloc[:,0].values)

In [12]:
exp_lung

Description,sample,WASH7P,OR4F5,CICP27,WASH9P,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,...,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-TE,MT-CYB
GTEX-111CU-0326-SM-5GZXO,GTEX-111CU-0326-SM-5GZXO,1.73487,0.015715,0.404045,3.44562,13.60320,701.845,36.4011,7.73125,21.4650,...,20639.6,30316.0,19533.3,9311.24,11979.60,25032.7,9814.89,13675.40,41.73750,17064.50
GTEX-111FC-1126-SM-5GZWU,GTEX-111FC-1126-SM-5GZWU,6.03857,0.023205,0.105289,10.74130,8.65547,327.442,22.0995,8.86811,15.4709,...,12900.5,20179.0,15523.2,9339.42,5804.09,15797.2,2139.15,1751.61,2.64138,11498.20
GTEX-111VG-0726-SM-5GIDC,GTEX-111VG-0726-SM-5GIDC,4.28893,0.052214,0.013162,7.43661,6.73684,344.753,21.8252,9.70944,13.8678,...,10059.8,15608.9,16461.0,6870.79,5435.81,14352.3,1982.03,1643.66,1.32074,7360.06
GTEX-111YS-0626-SM-5GZXV,GTEX-111YS-0626-SM-5GZXV,1.78961,0.036312,0.274594,4.59330,9.58309,447.230,36.2002,10.60700,13.8758,...,16867.4,24601.0,15304.7,8526.80,9350.72,22356.4,8367.10,14360.80,37.88790,13116.10
GTEX-1122O-0126-SM-5GICA,GTEX-1122O-0126-SM-5GICA,2.28437,0.052671,0.451412,9.01519,10.13190,461.800,58.8876,12.87930,21.1265,...,13476.1,25765.8,18491.4,9355.20,8347.75,25928.6,9148.45,19435.40,80.60410,17170.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTEX-ZYVF-1726-SM-5E443,GTEX-ZYVF-1726-SM-5E443,6.41811,0.119839,0.196351,9.02066,7.44990,428.987,28.7973,19.65260,30.5295,...,14148.1,23474.4,17661.8,12562.30,6679.04,14687.2,2572.85,2220.38,5.30477,10697.40
GTEX-ZYW4-1526-SM-5MR8B,GTEX-ZYW4-1526-SM-5MR8B,1.85024,0.000000,0.027833,10.41520,8.41835,542.612,34.0743,8.89207,16.7579,...,19272.4,28602.5,30368.7,8721.35,12027.80,28060.3,3061.72,2837.51,9.07718,18422.30
GTEX-ZYY3-0926-SM-5E454,GTEX-ZYY3-0926-SM-5E454,3.78499,0.017143,0.064817,6.68208,12.54690,519.206,64.5219,13.70470,31.7778,...,16571.0,24585.0,24884.4,10348.20,12210.50,28634.7,8225.72,13104.80,41.62710,14330.10
GTEX-ZZPT-1326-SM-5E43H,GTEX-ZZPT-1326-SM-5E43H,3.75378,0.051931,0.039271,12.84620,14.61880,384.816,64.4967,18.70530,43.0666,...,18875.3,29849.9,18260.5,12099.00,10579.80,24195.9,3508.40,2493.55,3.28394,16645.40


In [13]:
exp_breast = exp_breast.set_index(exp_breast.columns[0])
exp_breast = exp_breast.T 
exp_breast.insert(0, "sample", exp_breast. index)
#assert all(exp_breast.iloc[:, 0].values == meta_breast.iloc[:,0].values)

In [14]:
exp_breast

Description,sample,WASH7P,OR4F5,ENSG00000238009,ENSG00000268903,ENSG00000269981,ENSG00000241860,WASH9P,ENSG00000228463,ENSG00000237094,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-TE,MT-CYB,MT-TP
GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2826-SM-5GZXL,9.70327,0.018298,0.000000,2.093780,4.048150,0.114070,7.16661,0.174983,0.205100,...,41003.6,51020.1,14202.3,14019.80,35322.9,2929.78,1756.72,2.08274,20157.3,2.113370
GTEX-111YS-1926-SM-5GICC,GTEX-111YS-1926-SM-5GICC,3.98499,0.080215,0.040837,9.123300,19.669300,0.188916,5.65219,0.290973,0.812822,...,45532.2,34797.2,19903.3,18822.50,45468.6,11130.40,16237.40,46.87020,28117.2,0.617656
GTEX-1122O-1226-SM-5H113,GTEX-1122O-1226-SM-5H113,3.21666,0.046179,0.000000,6.138290,8.939620,0.101295,6.83673,0.071066,0.338185,...,40497.1,34161.2,15853.1,15896.10,38314.4,12135.90,18616.10,74.17390,28111.5,0.592636
GTEX-117XS-1926-SM-5GICO,GTEX-117XS-1926-SM-5GICO,5.03139,0.087630,0.097587,3.456410,8.986740,0.136574,8.14938,1.343720,0.554971,...,51302.6,48710.8,27211.2,22198.10,49521.9,5316.42,3714.49,11.63700,37297.5,5.060610
GTEX-117YX-1426-SM-5H12H,GTEX-117YX-1426-SM-5H12H,1.54279,0.053359,0.022637,3.885500,9.017730,0.277205,2.56654,0.158362,0.438609,...,49107.8,37807.0,20721.8,19287.70,48447.7,7501.45,10884.30,21.59500,28662.5,4.108610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTEX-ZYT6-0126-SM-5E45J,GTEX-ZYT6-0126-SM-5E45J,3.78941,0.054147,0.172285,1.642870,2.121350,0.187533,7.40743,0.705297,0.212427,...,50339.5,64436.2,40439.5,15623.80,42900.1,2554.60,1901.69,1.54082,37696.4,1.042320
GTEX-ZYW4-0826-SM-5GIDG,GTEX-ZYW4-0826-SM-5GIDG,2.47676,0.039261,0.024984,7.147330,9.952810,0.088385,6.95291,0.161836,0.308057,...,37674.5,68601.8,12704.1,13782.30,37327.9,3179.62,1721.12,2.23446,22142.6,2.267320
GTEX-ZYY3-2526-SM-GMXAZ,GTEX-ZYY3-2526-SM-GMXAZ,3.70044,0.025950,0.154130,12.417800,17.064300,0.236676,9.40208,0.228200,0.325786,...,22409.3,40051.6,16416.3,9668.92,18264.7,2486.48,2116.39,8.20509,14745.6,1.665150
GTEX-ZZ64-1226-SM-5E43R,GTEX-ZZ64-1226-SM-5E43R,2.39602,0.092476,0.016814,0.274855,0.730689,0.036604,5.86748,0.230893,0.290240,...,55658.6,35432.5,39532.0,23792.30,46090.1,10345.60,12679.10,47.11710,37514.9,2.034470


Run DysRegNet

In [16]:
data=dysregnet.run(expression_data= exp_lung, 
                   GRN="lung",
                   meta = meta_lung,
                   direction_condition=False,
                   normaltest=True,
                   R2_threshold=.2)

Processing lung GRN file...
GRN file and model ZIP already exist locally. Skipping download.
Extracting lung model ZIP...
Extracted models/lung_models.zip to models
You did not input any covariates in CatCov or ConCov parameters, proceeding without them.


Processing edges: 100000it [23:34, 70.68it/s]

Ratio of found models:  1.0
Skipped models:  1.0
Edges keys and lengths:
patient id: 604
Edges keys and lengths before normalization:
patient id: 604


In [16]:
data.get_results()

""
patient id
GTEX-111CU-0326-SM-5GZXO
GTEX-111FC-1126-SM-5GZWU
GTEX-111VG-0726-SM-5GIDC
GTEX-111YS-0626-SM-5GZXV
GTEX-1122O-0126-SM-5GICA
...
GTEX-ZYVF-1726-SM-5E443
GTEX-ZYW4-1526-SM-5MR8B
GTEX-ZYY3-0926-SM-5E454


In [17]:
# Load the pickle file
with open("pickle_models/AHR_FOS.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [18]:
print("Attributes and methods of the loaded object:")
print(dir(loaded_model))

Attributes and methods of the loaded object:
['HC0_se', 'HC1_se', 'HC2_se', 'HC3_se', '_HCCM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abat_diagonal', '_cache', '_data_attr', '_data_in_cache', '_get_robustcov_results', '_get_wald_nonlinear', '_is_nested', '_transform_predict_exog', '_use_t', '_wexog_singular_values', 'aic', 'bic', 'bse', 'centered_tss', 'compare_f_test', 'compare_lm_test', 'compare_lr_test', 'condition_number', 'conf_int', 'conf_int_el', 'cov_HC0', 'cov_HC1', 'cov_HC2', 'cov_HC3', 'cov_kwds', 'cov_params', 'cov_type', 'df_model', 'df_resid', 'eigenvals', 'el_test', 'ess', 'f_pvalue', 'f_test', 'fittedvalues', 'fvalue', 'get_influence', 'get_prediction',

In [19]:
print("Variables and their values in the loaded object:")
print(vars(loaded_model))

Variables and their values in the loaded object:
{'_results': <statsmodels.regression.linear_model.OLSResults object at 0x1789e5690>, '__doc__': "\n    Results class for for an OLS model.\n\n    Parameters\n    ----------\n    model : RegressionModel\n        The regression model instance.\n    params : ndarray\n        The estimated parameters.\n    normalized_cov_params : ndarray\n        The normalized covariance parameters.\n    scale : float\n        The estimated scale of the residuals.\n    cov_type : str\n        The covariance estimator used in the results.\n    cov_kwds : dict\n        Additional keywords used in the covariance specification.\n    use_t : bool\n        Flag indicating to use the Student's t in inference.\n    **kwargs\n        Additional keyword arguments used to initialize the results.\n\n    See Also\n    --------\n    RegressionResults\n        Results store for WLS and GLW models.\n\n    Notes\n    -----\n    Most of the methods and attributes are inherit

In [20]:
print(loaded_model.params)  # Coefficients
print(loaded_model.summary())  # Full model summary

const          0.000360
AHR            0.323355
gender_MALE   -0.000850
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     3.268
Date:                Wed, 22 Jan 2025   Prob (F-statistic):             0.0454
Time:                        14:38:04   Log-Likelihood:                -80.461
No. Observations:                  59   AIC:                             166.9
Df Residuals:                      56   BIC:                             173.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [21]:
# get the patient-specific dysregulate networks
# results = data.get_results()

# or with binary edges
results = data.get_results_binary()

# get R2 values, coefficients, and coefficient p-values for all models/edges
# data.get_model_stats()

In [ ]:
edge_counts = results.sum()
most_frequent_edge = edge_counts.idxmax()
max_count = edge_counts.max()

print (f"The most frequent edge is {most_frequent_edge} with {max_count} occurrences.")

ValueError: attempt to get argmax of an empty sequence